# Neural Septuple

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
class neural_septuple:

  def __init__(self, num_input_vec, num_output_vec):

    self.bias = True
    
    self.num_input_vec = num_input_vec
    self.num_output_vec = num_output_vec
    self.hidden_layer = 2
    self.num_hidden_layer_1_vec = 8
    self.num_hidden_layer_2_vec = 1
    #self.num_hidden_layer_3_vec = 2
    #self.num_hidden_layer_4_vec = 2
    self.num_total_vec = self.num_input_vec + self.num_output_vec + self.num_hidden_layer_1_vec + self.num_hidden_layer_2_vec #+ self.num_hidden_layer_3_vec #+ self.num_hidden_layer_4_vec
    self.idx_layer = [self.num_input_vec,8,1,self.num_output_vec]
    #self.state_vec = torch.tensor([5,6,0,0,0,0,7]).unsqueeze(dim=0).T
    
    p_in = torch.eye(self.num_input_vec)
    p_out = torch.eye(self.num_output_vec)
    p1 = torch.zeros((self.num_total_vec, self.num_total_vec))
    p2 = torch.zeros((self.num_total_vec, self.num_total_vec))

    p1[:self.num_input_vec, :self.num_input_vec] = p_in
    p2[-self.num_output_vec: , -self.num_output_vec:] = p_out
    self.p_in = p1 
    self.p_out = p2 
    I = torch.eye(self.num_total_vec)
    self.p_bulk = I - (self.p_in + self.p_out)
    self.p = self.p_in + self.p_out

  def create_state_vec(self,x,y):
    x = x.unsqueeze(dim=1)
    y = y.unsqueeze(dim=1)
    state = torch.zeros((sum(self.idx_layer), 1))
    state[:len(x)] = x
    state[-len(y):] = y
    self.state_vec = state
    self.input_vec = self.p_in @ self.state_vec.type(torch.float)
    self.output_vec = self.p_out @ self.state_vec.type(torch.float)


  def create_weight_matrix(self):
    count = 0
    self.final = []
    self.vec_idx = torch.tensor([i for i in range(self.num_total_vec)])
    for i in self.idx_layer:
      l = []
      for j in range(count, count+i):
        l.append(self.vec_idx[j])
      count += i
      self.final.append(l)
    self.weights = torch.zeros((self.num_total_vec, self.num_total_vec))
    i, j = 0,1
    while j < len(self.final):
      for row in self.final[i]:
        for col in self.final[j]:
          self.weights[col, row] = torch.randn((1,1))
      i += 1
      j += 1
    #self.weights[:, -1] = torch.randn((1,1))
    self.bias = torch.zeros((self.num_total_vec, 1))
    for i in range(self.num_input_vec, self.num_total_vec-self.num_output_vec):
        self.bias[i,0] = torch.randn((1,1))
    self.weights.requires_grad = True
    self.bias.requires_grad = True



  def feed_forword(self, x):
    first = x
    self.weights.requires_grad = True
    self.bias.requires_grad = True
    for i in range(self.hidden_layer):
      x = torch.tanh((self.weights @ x) + self.bias) + first
      #print(x)
    x = (self.weights @ x) + self.bias + first
    return x 
      


  def backword_feed(self, error):
    #print('LOSS: ', error)
    error.backward()
    for i in range(self.num_total_vec):
      for j in range(self.num_total_vec):
        if not self.weights[i,j]:
          self.weights.grad[i,j] = 0
    #print(self.weights.grad, '####')
    with torch.no_grad():
      self.weights = self.weights - (0.0001 * self.weights.grad)
      self.bias = self.bias - (0.0001 * self.bias.grad)
    #print(self.weights.grad, '****')
    if self.weights.grad:
      self.weights.grad.zero_()
    if self.bias.grad:
      self.bias.grad.zero_()


  def loss(self, pred, actual):
    return (pred - actual).T @ self.p_out @ (pred - actual)
        
    

  def create_projection_matrix(self, x):
    A = np.linalg.inv(x.T @ x)
    return x @ A @ x.T

In [ ]:
x = torch.randn(5000, 3)
y = torch.zeros((5000,1))
for i in range(len(x)):
  y[i] = ((x[i][0]**3) * 1.2 + (x[i][1]**2) * 2.3 +x[i][2] * 1)

In [ ]:
batch_size = 1
epochs = 100
a = neural_septuple(3,1)
loss = []
epoch = []

for i in range(epochs):
  print('EPOCHS: {} ***************'.format(i+1))
  count = 0
  error_count = 0
  for j in range(len(x)):
    a.create_state_vec(x[j],y[j])
    if i == 0 and j == 0:
      a.create_weight_matrix()
    pred = a.feed_forword(a.input_vec)
    error = (pred - a.state_vec).T @ a.p_out @ (pred - a.state_vec)
    #print(error)
    if (j+1) % batch_size == 0:
      error_count += error
      count += error
      error_count = error_count / batch_size
      #print('LOSS: ', error_count)
      a.backword_feed(error_count)
      error_count = 0
    else:
      with torch.no_grad():
        error_count += error
        count += error 
      #print(error_count)
  print('ERROR:{} '.format(i+1), count/5000)
  loss.append(count/5000)
  epoch.append(i)
      

EPOCHS: 1 ***************
ERROR:1  tensor([[35.5064]], grad_fn=<DivBackward0>)
EPOCHS: 2 ***************
ERROR:2  tensor([[27.6235]], grad_fn=<DivBackward0>)
EPOCHS: 3 ***************
ERROR:3  tensor([[22.0227]], grad_fn=<DivBackward0>)
EPOCHS: 4 ***************
ERROR:4  tensor([[18.6134]], grad_fn=<DivBackward0>)
EPOCHS: 5 ***************
ERROR:5  tensor([[16.6140]], grad_fn=<DivBackward0>)
EPOCHS: 6 ***************
ERROR:6  tensor([[15.0346]], grad_fn=<DivBackward0>)
EPOCHS: 7 ***************
ERROR:7  tensor([[13.6392]], grad_fn=<DivBackward0>)
EPOCHS: 8 ***************
ERROR:8  tensor([[12.5113]], grad_fn=<DivBackward0>)
EPOCHS: 9 ***************
ERROR:9  tensor([[11.6155]], grad_fn=<DivBackward0>)
EPOCHS: 10 ***************
ERROR:10  tensor([[10.8767]], grad_fn=<DivBackward0>)
EPOCHS: 11 ***************
ERROR:11  tensor([[10.2538]], grad_fn=<DivBackward0>)
EPOCHS: 12 ***************
ERROR:12  tensor([[9.7133]], grad_fn=<DivBackward0>)
EPOCHS: 13 ***************
ERROR:13  tensor([[9

In [ ]:
batch_size = 1
epochs = 100
a = neural_septuple(3,1)
loss = []
epoch = []

for i in range(epochs):
  print('EPOCHS: {} ***************'.format(i+1))
  count = 0
  error_count = 0
  for j in range(len(x)):
    a.create_state_vec(x[j],y[j])
    if i == 0 and j == 0:
      a.create_weight_matrix()
    pred = a.feed_forword(a.input_vec)
    error = (pred - a.state_vec).T @ a.p_out @ (pred - a.state_vec)
    #print(error)
    if (j+1) % batch_size == 0:
      error_count += error
      count += error
      error_count = error_count / batch_size
      #print('LOSS: ', error_count)
      a.backword_feed(error_count)
      error_count = 0
    else:
      with torch.no_grad():
        error_count += error
        count += error 
      #print(error_count)
  print('ERROR:{} '.format(i+1), count/5000)
  loss.append(count/5000)
  epoch.append(i)
      

EPOCHS: 1 ***************
ERROR:1  tensor([[29.9142]], grad_fn=<DivBackward0>)
EPOCHS: 2 ***************
ERROR:2  tensor([[24.4103]], grad_fn=<DivBackward0>)
EPOCHS: 3 ***************
ERROR:3  tensor([[21.3143]], grad_fn=<DivBackward0>)
EPOCHS: 4 ***************
ERROR:4  tensor([[18.8491]], grad_fn=<DivBackward0>)
EPOCHS: 5 ***************
ERROR:5  tensor([[16.6531]], grad_fn=<DivBackward0>)
EPOCHS: 6 ***************
ERROR:6  tensor([[14.5175]], grad_fn=<DivBackward0>)
EPOCHS: 7 ***************
ERROR:7  tensor([[12.3260]], grad_fn=<DivBackward0>)
EPOCHS: 8 ***************
ERROR:8  tensor([[10.3409]], grad_fn=<DivBackward0>)
EPOCHS: 9 ***************
ERROR:9  tensor([[8.8628]], grad_fn=<DivBackward0>)
EPOCHS: 10 ***************
ERROR:10  tensor([[7.5952]], grad_fn=<DivBackward0>)
EPOCHS: 11 ***************
ERROR:11  tensor([[6.6159]], grad_fn=<DivBackward0>)
EPOCHS: 12 ***************
ERROR:12  tensor([[5.9337]], grad_fn=<DivBackward0>)
EPOCHS: 13 ***************
ERROR:13  tensor([[5.38

In [ ]:
m = 1000
a.create_state_vec(x[m],y[m])

In [ ]:
pred = a.p_out @ a.feed_forword(a.input_vec)

In [ ]:
(pred - a.state_vec).T @ a.p_out @ (pred - a.state_vec)

tensor([[0.0141]], grad_fn=<MmBackward0>)

In [ ]:
a.output_vec[-1], pred[-1]

(tensor([2.5948]), tensor([2.7135], grad_fn=<SelectBackward0>))

In [ ]:
a.state_vec

tensor([[ 0.5135],
        [ 0.5786],
        [-0.5011],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.4315]])

In [ ]:
 pred 

tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.3120]], grad_fn=<MmBackward0>)

In [ ]:
pred

tensor([[ 0.7488],
        [ 0.7169],
        [-0.5766],
        [ 1.8890],
        [ 1.8890],
        [ 2.7780],
        [ 2.7780],
        [ 3.5561]], grad_fn=<AddBackward0>)

In [ ]:
a.loss(pred, a.state_vec)

tensor([[0.]], grad_fn=<MmBackward0>)

tensor([[1.0814]], grad_fn=<MmBackward0>)